In [1]:
import pandas as pd
df_final_sorted = pd.read_csv('result.csv')

with open('top_3.txt','w',encoding='utf-8') as f :
    for x in df_final_sorted.columns[4:]:
        df_copy = df_final_sorted.copy()

        if 'gk' in x:
            df_copy = df_copy[df_copy['position'] == 'GK']
        
        tmp = df_copy[['player',x]].copy()
        tmp.dropna(subset= [x], inplace = True)
        tmp = tmp.sort_values(by = x)

        f.write(f'---Top 3 highest for "{x}"---\n')
        for i,row in tmp.tail(3).iloc[::-1].iterrows():
            f.write(f"{row['player']}: {row[x]}\n")
        f.write('\n')

        f.write(f'--- Top 3 lowest for "{x}" ---\n')
        for i, row in tmp.head(3).iterrows():
            f.write(f"{row['player']}: {row[x]}\n")
        f.write('\n\n')

In [2]:
import pandas as pd
df = pd.read_csv('result.csv')

eliminated_cols = ['player','nationality','position','team','age']
numeric_df = df.drop(columns=eliminated_cols).apply(pd.to_numeric, errors='coerce')

numeric_df['team'] = df['team']

ovr_stats = numeric_df.drop(columns= 'team').agg(['median','mean','std']).T
ovr_stats.columns = ['Median','Mean','Std']
ovr_stats.index.name = 'Attribute'
ovr_stats.reset_index(inplace= True)

reshaped_ovr = pd.DataFrame({'Team' : ['all']})
for _,row in ovr_stats.iterrows() :
    reshaped_ovr[f"Median of {row['Attribute']}"] = [row['Median']]
    reshaped_ovr[f"Mean of {row['Attribute']}"] = [row['Mean']]
    reshaped_ovr[f"Std of {row['Attribute']}"] = [row['Std']]

team_stats = numeric_df.groupby('team').agg({'median','mean','std'})
team_stats.columns = [f"{stat.capitalize()} of {col}" for col, stat in team_stats.columns]
team_stats.reset_index(inplace=True)
team_stats.rename(columns={'team': 'Team'}, inplace=True)


final_df = pd.concat([reshaped_ovr, team_stats], ignore_index=True)
final_df.to_csv("results2.csv", index=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18728\1782549935.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_ovr[f"Median of {row['Attribute']}"] = [row['Median']]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18728\1782549935.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_ovr[f"Mean of {row['Attribute']}"] = [row['Mean']]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18728\1782549935.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

os.makedirs("./image/2.3", exist_ok=True)
df = pd.read_csv("result.csv")

selected_columns = ['gca', 'shots_on_target_per90', 'passes_into_penalty_area',
                    'tackles', 'interceptions', 'blocks']

for x in selected_columns:
    df_copy = df.copy()

    tmp = df_copy[x]
    tmp.dropna(axis=0, inplace=True)

    plt.figure(figsize=(5, 3))
    plt.hist(tmp, bins=15)
    plt.title(f"Distribution of {x}")
    plt.xlabel(x)
    plt.ylabel('Frequency')
    plt.savefig(f"./image/2.3/Distribution of {x}.png", dpi=300)
    plt.close()


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

os.makedirs("./image/2.3.1", exist_ok=True)

df_final_sorted = pd.read_csv('result.csv')

# 3 chỉ số tấn công + 3 chỉ số phòng thủ
selected_columns = ['gca', 'shots_on_target_per90', 'passes_into_penalty_area',
                    'tackles', 'interceptions', 'blocks']

for team_name in df_final_sorted['team'].unique():
    df_team = df_final_sorted[df_final_sorted['team'] == team_name]

    for col in selected_columns:
        tmp = df_team[col].dropna()

        if tmp.empty:
            continue 

        plt.figure(figsize=(5, 3))
        plt.hist(tmp, bins=15)
        plt.title(f'Distribution of {col} ({team_name})')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.tight_layout()

       
        safe_team_name = team_name.replace('/', '_').replace('\\', '_').replace(':', '_')
        plt.savefig(f"./image/2.3.1/Distribution of {col} of players in {safe_team_name}.png", dpi=300)
        plt.close()


In [ ]:
import pandas as pd

df = pd.read_csv("results2.csv")
df_teams = df[df['Team'] != 'all'].copy()
mean_cols = [col for col in df_teams.columns if col.startswith("Mean of")]

top_teams = []

for col in mean_cols:
    top_row = df_teams.loc[df_teams[col].idxmax()]
    top_teams.append({
        'Statistic': col.replace("Mean of ", ""),
        'Top Team': top_row['Team'],
        'Value': top_row[col]
    })

df_top = pd.DataFrame(top_teams)
print(df_top)

top_counts = df_top['Top Team'].value_counts()
print(top_counts.to_string())

best_team = top_counts.idxmax()
print(f"Best performing team based on overall means: {best_team}")


          Statistic         Top Team        Value
0             games  Nott'ham Forest    23.333333
1      games_starts        Brentford    17.600000
2           minutes        Brentford  1573.350000
3             goals        Liverpool     3.523810
4           assists        Liverpool     2.666667
..              ...              ...          ...
68          crosses           Fulham    31.863636
69  ball_recoveries        Brentford    68.650000
70      aerials_won        Brentford    26.600000
71     aerials_lost   Crystal Palace    25.666667
72  aerials_won_pct        Brentford    54.580000

[73 rows x 3 columns]
Top Team
Liverpool          23
Manchester City    11
Brentford          10
Crystal Palace      6
Nott'ham Forest     5
Bournemouth         5
Arsenal             3
Fulham              3
Chelsea             2
Newcastle Utd       2
Leicester City      1
Everton             1
Aston Villa         1
Best performing team based on overall means: Liverpool
